In [1]:
#magic commands
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#imports
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.makedirs('./objects', exist_ok=True)
import glob
import datetime
import functools
from pathlib import Path

import numpy as np
import cv2 as cv
import pandas as pd
from PIL import Image
import scipy
import scipy.ndimage
import scipy.special
import matplotlib.pyplot as plt

In [3]:
from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.data_classes import *
from lyft_dataset_sdk.utils.geometry_utils import *

In [4]:
data_path = Path('data')
train = data_path/'train_data'

In [5]:
classes = ["car", "motorcycle", "bus", "bicycle", "truck",\
           "pedestrian","other_vehicle", "animal", "emergency_vehicle"]

In [6]:
#LyftDataset??
data = LyftDataset(data_path = '.', json_path = str(train))

9 category,
18 attribute,
4 visibility,
18421 instance,
10 sensor,
148 calibrated_sensor,
177789 ego_pose,
180 log,
180 scene,
22680 sample,
189504 sample_data,
638179 sample_annotation,
1 map,
Done loading in 15.1 seconds.
Reverse indexing ...
Done reverse indexing in 3.2 seconds.


In [7]:
records = [(data.get('sample', record['first_sample_token'])['timestamp'], record) for record in data.scene]

In [9]:
entries = []
for ts, record in sorted(records):
    start = data.get('sample', record['first_sample_token'])['timestamp'] / 10000000
    token = record['token']
    name = record['name']
    date = datetime.datetime.utcfromtimestamp(start)
    first_sample_token = record['first_sample_token']
    car = "-".join(name.split("-")[:2])
    entries.append((car, name, date, token, first_sample_token))

In [10]:
cols = ['car', 'scene', 'date', 'scene_token', 'first_sample_token']
d = pd.DataFrame(entries, columns = cols)

In [11]:
car_count = d.groupby("car")['scene_token'].count()
car_count

car
host-a004    42
host-a005     1
host-a006     3
host-a007    26
host-a008     5
host-a009     9
host-a011    51
host-a012     2
host-a015     6
host-a017     3
host-a101    20
host-a102    12
Name: scene_token, dtype: int64